# <a href="https://onlinelibrary.wiley.com/doi/abs/10.3982/ECTA17434" target="__blank">Using the Sequence-Space Jacobian to Solve and Estimate Heterogeneous-Agent Models</a>

#### Adrien Auclert, Bence Bardóczy, Matt Rognlie, Ludwig Straub

Notebook created by Qingyuan Fang on Feb 20 2024

(Materials borrow heavily from the authors' slides)



## Summary

This paper proposes a fast, efficient algorithm to solve heterogeneous-agent (HA) models in GE with aggregate shocks. The algorithm consists of three steps:

1. Write HA model as a collection of **blocks** along a **directed acyclic graph (DAG)**
2. Compute the **Jacobian** of each block
3. Use Jacobians for: IRFs, determinacy, full-info estimation, nonlinear transitions, ...

<u>Predecessors</u> <br />
When idiosyncratic risk $\gg$ aggregate risk, 
1. [Reiter method] Linearize wrt aggregate shocks, solve **linear state space system**
2. [MIT shock method] Assume perfect foresight wrt aggregate shocks, solve nonlinear system in **sequence space**

<u>SSJ</u> directly solves **linear system** in the **sequence space** $\Rightarrow$​ Fast, accurate, modular, intuitive, accessible

<u>Restrictions</u>: Agents in the model can only interact via **limited set of aggregate variables**. SSJ doesn't apply to models where the behavior of HA depends on the anticipated future disctribution through the value function.

## Models as collections of blocks arranged along a DAG

### Block: Mapping from sequence of inputs to sequence of outputs

**Example 1**: heterogeneous household block $\left\{r_{t}, w_{t}\right\} \rightarrow\left\{C_{t}\right\}$​

- Exogenous Markov chain for skills $\Pi\left(e^{\prime} \mid e\right)$
- Households

$$
\begin{aligned}
\max \ & \mathbb{E}_{0} \sum_{t} \beta^{t} u\left(c_{i t}\right) \\
c_{i t}+k_{i t} & \leq\left(1+r_{t}\right) k_{i t-1}+w_{t} e_{i t} \\
k_{i t} & \geq 0
\end{aligned}
$$

Given initial distribution $D_{0}(e, k_{-})$, path of aggregate consumption $C_{t} \equiv \int C_{t} (e, k_{-})$ $D_{t} (e, d k_{-} )$ only depends on $\left\{r_{s}, w_{s}\right\}_{s=0}^{\infty}$. 

**Example 2**: representative firm block with $L=1\left\{K_{t}, Z_{t}\right\} \rightarrow\left\{Y_{t}, I_{t}, r_{t}, w_{t}\right\}$
$$
\begin{aligned}
Y_{t} & =Z_{t} K_{t-1}^{\alpha} \\
I_{t} & =K_{t}-(1-\delta) K_{t-1} \\
r_{t} & =\alpha Z_{t} K_{t-1}^{\alpha-1}-\delta \\
w_{t} & =(1-\alpha) Z_{t} K_{t-1}^{\alpha}
\end{aligned}
$$

Given initial capital $K_{-1}$, path of $\left\{Y_{t}, I_{t}, r_{t}, W_{t}\right\}_{t=0}^{\infty}$ only depends on $\left\{K_{s}, Z_{s}\right\}_{s=0}^{\infty}$​.

**Example 3**: goods market clearing block $\left\{Y_{t}, C_{t}, I_{t}\right\} \rightarrow\left\{H_{t} \equiv C_{t}+I_{t}-Y_{t}\right\}$​


### Model: Set of blocks, arranged along a directed acyclic graph (DAG)

- some inputs are exogenous shocks, e.g. $\left\{Z_{t}\right\}$
- some inputs are endogenous unknowns, e.g. $\left\{K_{t}\right\}$
- some outputs are target sequences that must equal zero in GE, e.g. $\left\{H_{t}\right\}$​​​​

There must be as many targets as unknowns to close the model.

**Example 1:** Krussell-Smith

Combining the three "blocks" from the previous subsection, we can obtain the following DAG:

![](./Figures/DAG_KS.jpg)

, which can be collapsed into mapping

$$
H_{t}\left(\left\{K_{s}\right\},\left\{Z_{s}\right\}\right)=C_{t}+I_{t}-Y_{t}
$$

The GE path of $\left\{K_{s}\right\}$ achieves $H_{t}\left(\left\{K_{s}\right\},\left\{Z_{s}\right\}\right)=0$

**Example 2**: Krussell-Smith with endogenous labor

![](./Figures/DAG_KS2.jpg)

DAG can be collapsed into mapping

$$
\mathbf{H}_{t}\left(\left\{K_{s}, L_{s}\right\},\left\{Z_{s}\right\}\right)=\left\{C_{t}+I_{t}-Y_{t}, N_{s}-L_{s}\right\}
$$

The GE path of $\left\{K_{s}, L_{s}\right\}$ achieves $\mathbf{H}_{t}\left(\left\{K_{s}, L_{s}\right\},\left\{Z_{s}\right\}\right)=0$

**Example 3**: Simple one-asset HANK model with sticky wages

![](./Figures/DAG_OneAsset.jpg)

**Example 4**: A complicated two-asset HANK model

![](./Figures/DAG_TwoAsset.jpg)

## Compute the **Jacobian** of each block
A block **Jacobian** is the derivatives of its outputs wrt its inputs.

**Block Jacobians for the Krussell-Smith model**

- household block: $\left\{\frac{\partial C_{t}}{\partial w_{s}}\right\},\left\{\frac{\partial C_{t}}{\partial r_{s}}\right\} \rightsquigarrow \operatorname{denoted \ as} \mathcal{J}^{\mathcal{C}, w}, \mathcal{J}^{\mathcal{C}, r}$

- firm block: $\left\{\frac{\partial w_{t}}{\partial K_{s}}\right\},\left\{\frac{\partial w_{t}}{\partial Z_{s}}\right\},\left\{\frac{\partial r_{t}}{\partial K_{s}}\right\},\left\{\frac{\partial r_{t}}{\partial Z_{s}}\right\}, \ldots \rightsquigarrow \operatorname{denoted \ as} \mathcal{J}^{w, K}, \mathcal{J}^{w, z}, \mathcal{J}^{r, K}, \mathcal{J}^{r, z}, \ldots$

- We can then apply the **chain rule** to get the Jacobians of H: 

$$\frac{\partial \mathbf{H}}{\partial \mathbf{K}}=\mathcal{J}^{C, r} \mathcal{J}^{r, K}+\mathcal{J}^{C, w} \mathcal{J}^{w, K}+\mathcal{J}^{l, K}-\mathcal{J}^{Y, K}$$

$$\frac{\partial \mathbf{H}}{\partial \mathbf{Z}}=\mathcal{J}^{C, r} \mathcal{J}^{r, Z}+\mathcal{J}^{C, w} \mathcal{J}^{w, Z}+\mathcal{J}^{l, Z}-\mathcal{J}^{Y, Z}$$

**From block Jacobians to impulse responses**

Suppose shock is $d \mathbf{Z}=\left\{d Z_{t}\right\}$ [with $d Z_{t}=0, t \geq T_{0}$ ] Since $\mathbf{H}(\mathbf{K}, \mathbf{Z})=0$ after the shock, we can solve for unknown $d \mathbf{K}$ as:

$$
d \mathbf{K}=-\left(\frac{\partial \mathbf{H}}{\partial \mathbf{K}}\right)^{-1} \frac{\partial \mathbf{H}}{\partial \mathbf{Z}} d \mathbf{Z}
$$

Then, we can use Jacobians to back out any IRF of interest, e.g. IRF of output

$$
d \mathbf{Y}=\mathcal{J}^{Y, K} d \mathbf{K}+\mathcal{J}^{Y, Z} d \mathbf{Z}
$$

**Claim: block Jacobians are sufficient to obtain all GE impulse responses**.

**Aggregate shocks and $MA(\infty)$ representation**

Certainty equivalence implies that $d\mathbf{K}$ is also the $M A(\infty)$ representation in models with aggregate shocks:
Suppose $\left\{d \tilde{Z}_{t}\right\}$ is $M A(\infty)$ in iid structural innovation vectors $\left\{\epsilon_{t}\right\}$ :

$$
d \tilde{Z}_{t}=\sum_{s=0}^{\infty} d Z_{s} \epsilon_{t-s}
$$

then,

$$
d \tilde{K}_{t}=\sum_{s=0}^{\infty} d K_{s} \epsilon_{t-s}
$$

Applications:

1. Simulation method
2. Analytical second moments for any $X, Y: \operatorname{Cov}\left(d \tilde{X}_{t}, d \tilde{Y}_{t^{\prime}}\right)=\sigma_{\epsilon}^{2} \sum_{s=0}^{T-\left(t^{\prime}-t\right)} d X_{s} d Y_{s+t^{\prime}-t}$
3. Model estimation <br />
Let $\mathbf{V}(\theta)$ be the covariance matrix for a set of $k$ outputs, where $\theta \equiv$ parameters. Assuming Gaussian innovations, the log-likelihood of observed data $\mathbf{Y}$ given $\theta$ is:

$$
\mathcal{L}(\mathbf{Y} ; \theta)=-\frac{1}{2} \log \operatorname{det} \mathbf{V}(\theta)-\frac{1}{2} \mathbf{Y}^{\prime} \mathbf{V}(\theta)^{-1} \mathbf{Y}
$$ 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;note that we can reuse the same Jacobians for any $d \mathbf{Z}$ as long as there is no need to recalculate HA steady state.


## Speeding up HA Jacobian computation

### Direct algorithm

Assume initial condition is s.s., with $r_{t}=r, w_{t}=w, D_{0}\left(e, k_{-}\right)=D\left(e, k_{-}\right)$. We want to know $\mathcal{J}_{t, s} \equiv \frac{\partial C_{t}}{\partial w_{s}}$ for $s, t \in\{0, \ldots, T-1\}$
1. perturb $w_{s} \equiv w+\epsilon$
2. iterate backwards to get perturbed policies: $\mathbf{c}_{t}^{s}\left(e, k_{-}\right), \mathbf{k}_{t}^{s}\left(e, k_{-}\right)$
3. iterate forward to get perturbed distributions $D_{t}^{s}\left(e, k_{-}\right)$
4. put together to get perturbed aggregate consumption: $C_{t}^{s}=\int \mathbf{c}_{t}^{s}\left(e, k_{-}\right) D_{t}^{s}\left(e, d k_{-}\right)$
5. compute $\mathcal{J}$ from $\mathcal{J}_{t, s} \equiv\left(C_{t}^{s}-C\right) / \epsilon$

This algorithm is slow because the steps above need to be done $T$ times, once for each s.

### Fake news algorithm

The paper proposes fake news algorithm, which only requires single backward iteration \& single forward iteration. The key idea is to exploit the time symmetries around the steady-state.

**The fake news matrix**

Recall the Jacobian
$$
\mathcal{J}=\left(\begin{array}{cccc}
\mathcal{J}_{00} & \mathcal{J}_{01} & \mathcal{J}_{02} & \cdots \\
\mathcal{J}_{10} & \mathcal{J}_{11} & \mathcal{J}_{12} & \cdots \\
\mathcal{J}_{20} & \mathcal{J}_{12} & \mathcal{J}_{22} & \cdots \\
\vdots & \vdots & \vdots & \ddots
\end{array}\right)
$$
We can think of $\mathcal{J}$ as a **news matrix**. <br />
Intuition: the $s$-th column of $\mathcal{J} \equiv\left(\frac{\partial C_{t}}{\partial w_{s}}\right)$ represents the response to news that a shock will hit in period $s$.

Define a auxiliary matrix $\mathcal{F}$:

$$
\mathcal{F}_{t, s} \equiv \begin{cases}\frac{\partial C_{t}}{\partial W_{s}} & s=0 \text { or } t=0 \\ \frac{\partial C_{t}}{\partial W_{s}}-\frac{\partial C_{t-1}}{\partial W_{s-1}} & s, t>0\end{cases}
$$
i.e.
$$
\mathcal{F}=\left(\begin{array}{cccc}
\mathcal{J}_{00} & \mathcal{J}_{01} & \mathcal{J}_{02} & \cdots \\
\mathcal{J}_{10} & \mathcal{J}_{11}-\mathcal{J}_{00} & \mathcal{J}_{12}-\mathcal{J}_{01} & \cdots \\
\mathcal{J}_{20} & \mathcal{J}_{12}-\mathcal{J}_{10} & \mathcal{J}_{22}-\mathcal{J}_{11} & \cdots \\
\vdots & \vdots & \vdots & \ddots
\end{array}\right)
$$

We can think of $\mathcal{F}$ as the **fake news matrix**. <br />
Intuition: at $t=0$, news comes that a period $s$ shock will hit $\rightarrow \frac{\partial C_{0}}{\partial w_{s}}$; at $t=1$, that news is withdrawn $\rightarrow \frac{\partial C_{1}}{\partial w_{s}}-\frac{\partial C_{0}}{\partial w_{s-1}}$

We can recover $\mathcal{J}$ from $\mathcal{F}$ by adding elements from top left diagonal. <br />
Intuition: a news shock is equivalent to a sequence of fake news shocks.



**Single backward iteration**

**Claim: Single backward iteration is enough to recover $\mathbf{c}_{t}^{s}\left(e, k_{-}\right), \mathbf{k}_{t}^{s}\left(e, k_{-}\right)$**

Intuition: only the time $s-t$ until the perturbation matters

$$
\mathbf{c}_{t}^{s}\left(e, k_{-}\right)= \begin{cases}\mathbf{c}\left(e, k_{-}\right) & s<t \\ \mathbf{c}_{T-1-(s-t)}^{T-1}\left(e, k_{-}\right) & s \geq t\end{cases}
$$

- Therefore, we only need a single backward iteration with $s=T-1$ to get all the $\mathbf{c}_{t}^{s}$
- From these we can obtain:
    - $C_{\mathrm{0}}^{\mathrm{s}}=\int \mathbf{c}_{\mathrm{0}}^{\mathrm{s}}\left(e, k_{-}\right) D\left(e, d k_{-}\right)$ $\rightarrow$ first row of Jacobian $\mathcal{J}_{\mathrm{0s}}=\frac{\partial C_{\mathrm{0}}}{\partial \mathrm{w}_{\mathrm{s}}}=\mathcal{F}_{\mathrm{0s}}$
    - $D_{1}^{s}\left(e, d k_{-}\right)$, distributions at date 1 implied by new policy $\mathbf{c}_{0}^{s}$ at date 0

**Single forward iteration**

- Iterate the distributions forward using s.s. policies 

$$
\mathbf{D}_{t}^{s}\left(e, d k_{-}\right)=\left(\Lambda^{\prime}\right)^{t-1} \mathbf{D}_{1}^{s}\left(e, d k_{-}\right)
$$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;, where $\Lambda$ is s.s. transition matrix
- Construct aggregate consumption using s.s. policies

$$
C_{t}^{s} \equiv \int \mathbf{c}\left(e, k_{-}\right) D_{t}^{s}\left(e, d k_{-}\right) \quad \Rightarrow \quad C_{t}^{s}=\mathbf{c}^{\prime}\left(\Lambda^{\prime}\right)^{t-1} \mathbf{D}_{1}^{s}
$$

- Fake news matrix $\mathcal{F}_{t, s}=\left(C_t^s-C\right) / \epsilon \Rightarrow$ block Jacobian $\mathcal{J}$ 

## References

1. Auclert, Adrien, et al. "Using the sequence‐space Jacobian to solve and estimate heterogeneous‐agent models." Econometrica 89.5 (2021): 2375-2408.
2. Boppart, Timo, Per Krusell, and Kurt Mitman. "Exploiting MIT shocks in heterogeneous-agent economies: the impulse response as a numerical derivative." Journal of Economic Dynamics and Control 89 (2018): 68-92.
3. Reiter, Michael. "Solving heterogeneous-agent models by projection and perturbation." Journal of Economic Dynamics and Control 33.3 (2009): 649-665.